<a href="https://colab.research.google.com/github/AyeshaIjazTabassum/PythonAIBootcamp/blob/main/Day27.5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Conditional Chaining

In [10]:
! pip install python-dotenv langchain langchain-core langchain-google-genai

In [3]:
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables import RunnableLambda,RunnableBranch
from google.colab import userdata


GOOGLE_API_KEY = userdata.get('API')
load_dotenv()
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite", api_key=GOOGLE_API_KEY)

In [4]:
positive_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human","Generate a thank you note for this positive feedback: {feedback}."),
    ]
)

negative_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human","Generate a response addressing this negative feedback: {feedback}."),
    ]
)

neutral_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        (
            "human","Generate a request for more details for this neutral feedback: {feedback}.",
        ),
    ]
)
escalate_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        (
            "human",
            "Generate a message to escalate this feedback to a human agent: {feedback}.",
        ),
    ]
)

In [5]:
classification_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human",
         "Classify the sentiment of this feedback as positive, negative, neutral, or escalate: {feedback}."),
    ]
)

In [6]:
branches = RunnableBranch(
    (
        lambda x: "positive" in x,
        positive_feedback_template | llm| StrOutputParser()  # Positive feedback chain
    ),
    (
        lambda x: "negative" in x,
        negative_feedback_template | llm | StrOutputParser()  # Negative feedback chain
    ),
    (
        lambda x: "neutral" in x,
        neutral_feedback_template | llm | StrOutputParser()  # Neutral feedback chain
    ),
    escalate_feedback_template | llm | StrOutputParser()
)

In [7]:
classification_chain = classification_template | llm | StrOutputParser()

In [8]:
chain = classification_chain | branches

* Run the chain with an example review
* Good review - "The product is excellent. I really enjoyed using it and found it very helpful."
* Bad review - "The product is terrible. It broke after just one use and the quality is very poor."
* Neutral review - "The product is okay. It works as expected but nothing exceptional."
* Default - "I'm not sure about the product yet. Can you tell me more about its features and benefits?"

In [9]:
review = "The product is terrible. It broke after just one use and the quality is very poor."
result = chain.invoke({"feedback": review})

print(result)

Here are a few options for escalating negative feedback to a human agent, ranging in formality and directness. Choose the one that best fits your context:

**Option 1 (Direct and Concise):**

> **Escalation needed: Negative feedback received.**

**Option 2 (Slightly More Detail):**

> **Urgent: Negative customer feedback requires immediate attention from a human agent.**

**Option 3 (More Formal):**

> **Escalation Request: A negative feedback submission has been received and requires review by a human agent.**

**Option 4 (If you have a specific reason for escalation):**

> **Escalation Required: Negative feedback received. [Briefly state the reason, e.g., "Potential issue with service," "Customer expressed significant dissatisfaction," "Urgent follow-up needed."]**

**Option 5 (If it's a system-generated alert):**

> **Alert: Negative feedback detected. Please route to a human agent for review.**

**Key elements to consider when choosing/adapting:**

*   **Urgency:** Does this need i